In [ ]:
# Drop rows with missing values in the specified columns


#fill Nan
numerical_columns = train_data.select_dtypes(include=['number']).columns  # Select only numeric columns
train_data[numerical_columns] = train_data[numerical_columns].fillna(train_data[numerical_columns].mean())
#fill train_data
new_columns = pd.DataFrame({
    'team_rest_diff': train_data['home_team_rest'] - train_data['away_team_rest'],
    'pitcher_rest_diff': train_data['home_pitcher_rest'] - train_data['away_pitcher_rest'],
    'batting_avg_diff': train_data['home_batting_batting_avg_10RA'] - train_data['away_batting_batting_avg_10RA'],
    'pitching_ERA_diff': train_data['home_pitching_earned_run_avg_10RA'] - train_data['away_pitching_earned_run_avg_10RA'],
    'onbase_perc' : train_data['home_batting_onbase_plus_slugging_10RA'] / train_data['away_batting_onbase_plus_slugging_10RA']
})

# Concatenate the new columns to the original DataFrame
train_data = pd.concat([train_data, new_columns], axis=1)
columns_to_drop = ['away_pitcher', 'home_pitcher', 'date','id', 'home_team_abbr','away_team_abbr','season','home_pitcher_rest','away_pitcher_rest','home_team_rest','away_team_rest','home_batting_batting_avg_10RA','away_batting_batting_avg_10RA','home_pitching_earned_run_avg_10RA','away_pitching_earned_run_avg_10RA','home_batting_onbase_plus_slugging_10RA','away_batting_onbase_plus_slugging_10RA']
train_data.drop(columns=columns_to_drop, inplace=True)
train_data['is_night_game'] = train_data['is_night_game'].apply(
    lambda x: np.random.choice([True, False]) if pd.isnull(x) else x
)

# Example of splitting the dataset by winning and losing teams
# Assuming 'home_team_win' is the column that indicates the outcome (1 = win, 0 = loss)
winning_team_data = train_data[train_data['home_team_win'] == 1]
losing_team_data = train_data[train_data['home_team_win'] == 0]
numeric_columns = train_data.select_dtypes(include=['number']).columns
# Loop through all columns and perform a t-test for each column
count = 0
add_colums=[]
for column in numeric_columns:
    if column != 'home_team_win':  # Skip the outcome column itself
        # Perform t-test (assuming data is numeric)
        t_stat, p_value = stats.ttest_ind(winning_team_data[column], losing_team_data[column], nan_policy='omit')
        
        # If p-value is less than 0.05, print the result
        if p_value < 0.05:
            # print(f"Significant variable: {column} | p-value: {p_value}")
            add_colums.append(column)
# One-hot encode 'home_team_season' and 'away_team_season'
original_columns = train_data.columns.tolist()

# One-hot encode 'home_team_season' and 'away_team_season'
season_columns = ['home_team_season', 'away_team_season']
train_data = pd.get_dummies(train_data, columns=season_columns, drop_first=True)

# Save the new column names after one-hot encoding
new_columns = train_data.columns.tolist()

# Identify the newly added columns
dummy_columns = [col for col in new_columns if col not in original_columns]

# Add these to your `add_columns` list
add_colums.extend(dummy_columns)

# View the resulting columns
print(train_data.columns)